## Get data from NID Japan

In [14]:
import requests
from bs4 import BeautifulSoup
import re
import os
from datetime import datetime

def extract_info(text, year):
    match = re.search(r'week (\d+)\（(\d+/\d+)～(\d+/\d+)\）', text)

    if match:
      start_date = match.group(2)
      start_date = datetime.strptime(str(year) + '/' + start_date, "%Y/%m/%d")
      end_date = match.group(3)
      end_date = datetime.strptime(str(year) + '/' + end_date, "%Y/%m/%d")
      return start_date, end_date
    else:
        return None, None

def extract_title(text):
    match = re.search(r'(\d{4}) week (\d+)', text)
    if match:
        year = match.group(1)
        week = match.group(2)
        return year, week
    else:
        return None, None

def get_jp_data(i):
  url = f"https://www.niid.go.jp/niid/en/survaillance-data-table-english.html?start={i}"

  response = requests.get(url)
  response.raise_for_status()

  soup = BeautifulSoup(response.text, 'html.parser')

  title = soup.find('h2').text
  title = re.sub(r'[\n\t]', '', title)
  year, week = extract_title(title)

  # date info is in the first paragraph
  date_info = soup.find('p').text
  start_date, end_date = extract_info(date_info, year)

  # get csv links
  csv_links = []
  for link in soup.find_all('a', href=True):
      href = link['href']
      if href.lower().endswith('.csv'):
          if not href.startswith('http'):
              href = requests.compat.urljoin(url, href)
          csv_links.append(href)

  # download the first csv file
  csv_url = csv_links[0]
  response = requests.get(csv_url)
  response.raise_for_status()

  # get file extension
  _, file_extension = os.path.splitext(csv_url)

  with open(f'JP/{title}{file_extension}', 'wb') as f:
      f.write(response.content)
  
  outcome = {
      "title": title,
      "year": year,
      "week": week,
      "start_date": start_date,
      "end_date": end_date,
      "path": f'JP/{title}.csv',
      "url": url
  }

  return outcome

In [15]:
get_jp_data(1)

{'title': 'IDWR Surveillance Data Table 2024 week 08',
 'year': '2024',
 'week': '08',
 'start_date': datetime.datetime(2024, 2, 19, 0, 0),
 'end_date': datetime.datetime(2024, 2, 25, 0, 0),
 'path': 'JP/IDWR Surveillance Data Table 2024 week 08.csv',
 'url': 'https://www.niid.go.jp/niid/en/survaillance-data-table-english.html?start=1'}

In [16]:
from multiprocessing import Pool
import multiprocessing

id_range = range(1, 604, 1)

num_processes = int(0.9 * multiprocessing.cpu_count())
with Pool(processes=num_processes) as pool:
    results = pool.map(get_jp_data, id_range)

## Reading Data

In [17]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [18]:
def clean_data(i):
  _, year, week, _, end_date, file_name, url = results[i].values()
  try:
    df = pd.read_csv(file_name, header= None, skiprows=3)
  except pd.errors.ParserError as e:
    print(f"Error reading {file_name}, i = {i}")
    data = {
      "Area": [None],
      "Date": [end_date],
      "Year": [year],
      "Week": [week],
      "Disease": [None],
      1: [None],
      2: [None],
      "Cases": [None],
      "URL": [url]
    }
    df = pd.DataFrame(data)
    return df

  # find the data start row
  for i in range(len(df)):
    values = df.iloc[i, 1]
    if re.match(r'^\d+$', str(values)):
      start_row = i
      break

  # column names
  df_names = df.iloc[:start_row]
  df_names = df_names.T
  df_names = df_names.reset_index()
  df_names['index'] = df_names['index']
  df_names = df_names.ffill(axis=0)

  # adjust the column names
  df.columns = range(df.shape[1])
  df = df.iloc[start_row:]
  df = df.melt(id_vars=[0], value_vars=range(1, df.shape[1]), var_name='State', value_name='Cases')
  df = df.rename(columns={0: 'Area'})

  # merge with df_names by index and State
  df = pd.merge(df, df_names, left_on='State', right_on='index', how='left')
  df = df.rename(columns={0: 'Disease'})
  # add column 2 if not present
  if 1 not in df.columns:
      df[1] = ''
  if 2 not in df.columns:
      df[2] = ''

  # add Date
  df['Date'] = end_date
  df['Year'] = year
  df['Week'] = week

  # add URL
  df['URL'] = url

  df = df[['Area', 'Date', 'Year', 'Week', 'Disease', 1, 2,  'Cases', 'URL']]

  return df

In [20]:
results[601]

{'title': 'IDWR Survaillance Data Table 2012 week 33',
 'year': '2012',
 'week': '33',
 'start_date': None,
 'end_date': None,
 'path': 'JP/IDWR Survaillance Data Table 2012 week 33.csv',
 'url': 'https://www.niid.go.jp/niid/en/survaillance-data-table-english.html?start=602'}

In [21]:
clean_data(11)

,Area,Date,Year,Week,Disease,1,2,Cases,URL
0,Total No.,2023-12-10,2023,49,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
1,Hokkaido,2023-12-10,2023,49,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
2,Aomori,2023-12-10,2023,49,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
3,Iwate,2023-12-10,2023,49,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
4,Miyagi,2023-12-10,2023,49,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
...,...,...,...,...,...,...,...,...,...
8347,Kumamoto,2023-12-10,2023,49,Multiple drug-resistant Acinetobacter infection,Cum 2023,,0,https://www.niid.go.jp/niid/en/survaillance-da...
8348,Oita,2023-12-10,2023,49,Multiple drug-resistant Acinetobacter infection,Cum 2023,,0,https://www.niid.go.jp/niid/en/survaillance-da...
8349,Miyazaki,2023-12-10,2023,49,Multiple drug-resistant Acinetobacter infection,Cum 2023,,0,https://www.niid.go.jp/niid/en/survaillance-da...
8350,Kagoshima,2023-12-10,2023,49,Multiple drug-resistant Acinetobacter infection,Cum 2023,,0,https://www.niid.go.jp/niid/en/survaillance-da...


In [22]:
from multiprocessing import Pool
import multiprocessing

id_range = range(0, len(results), 1)

num_processes = int(0.9 * multiprocessing.cpu_count())
with Pool(processes=num_processes) as pool:
    alldata = pool.map(clean_data, id_range)

final_df = pd.concat(alldata, ignore_index=True)

# save to csv
final_df.to_csv('./JP/AllData.csv', index=False)

Error reading JP/IDWR Survaillance Data Table 2012 week 33.csv, i = 601


/tmp/ipykernel_657921/3593706733.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(alldata, ignore_index=True)


In [23]:
# head of the data
final_df.head()


,Area,Date,Year,Week,Disease,1,2,Cases,URL
0,Total No.,2024-02-25,2024,08,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
1,Hokkaido,2024-02-25,2024,08,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
2,Aomori,2024-02-25,2024,08,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
3,Iwate,2024-02-25,2024,08,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...
4,Miyagi,2024-02-25,2024,08,Ebola hemorrhagic fever,Current week,,0,https://www.niid.go.jp/niid/en/survaillance-da...


In [24]:
# unique date
yw_df = final_df[['Year', 'Week']].drop_duplicates()
# reorder
yw_df = yw_df.sort_values(by=['Year', 'Week'])
# reset index
yw_df = yw_df.reset_index(drop=True)
yw_df['value'] = 1
# long table to wide table
yw_df = yw_df.pivot(index='Year', columns='Week', values='value')
# fill na with 0
yw_df = yw_df.fillna(0)
# save to csv
yw_df.to_csv('./JP/YearWeek.csv')

In [25]:
pertussis_df = final_df.dropna(subset=['Disease'])
# unique disease
disease_df = pertussis_df['Disease'].drop_duplicates()
# reset index
disease_df = disease_df.reset_index(drop=True)
print(disease_df)

0                               Ebola hemorrhagic fever
1                       Crimean-Congo hemorrhagic fever
2                                              Smallpox
3                      South American hemorrhagic fever
4                                                Plague
                             ...                       
102    \rSevere invasive streptococcal infections(TSLS)
103         \rAcquired immunodeficiency syndrome (AIDS)
104                                 2019-nCoV infection
105                            Meningococcal meningitis
106      Avian influenza (exclud. Avian influenza H5N1)
Name: Disease, Length: 107, dtype: object


In [ ]:
# Drop Disease is NA /Nan
pertussis_df = final_df.dropna(subset=['Disease'])
# Filter Disease contains pertussis
pertussis_df = pertussis_df[pertussis_df['Disease'].str.contains('Pertussis', case=False)]
pertussis_df = pertussis_df[pertussis_df[1].str.contains('Current', case=False)]
# filter Area is in Total
pertussis_df = pertussis_df[pertussis_df['Area'].isin(['Total No.'])]

# Arrange by Year and Week
pertussis_df = pertussis_df.sort_values(by=['Year', 'Week'])
# save to csv
pertussis_df.to_csv('./JP/pertussis.csv', index=False)